In [130]:
import random
import numpy as np 
 
# Initialisation de la population of 4 elements
def Initialiser_population(population_size):
    population = []
    for j in range(0, population_size):
        res = ""
        l = [i for i in range(0, 16)]
        for i in range(0, 16):
            elem = random.choice(l)
            bin_elem = f'{elem:04b}'
            res += str(bin_elem)
            l.remove(elem)
        population.append(res)
    return population
 
population = Initialiser_population(4)
print(population)

['1110110110111010001001110100000010011111010101101000000100111100', '0101101111110010000001110011111010000110101010010100110111000001', '0100000010111101111010010010000111111000001111000111011001011010', '1011101010000110100100111110000011000001001001111101010001011111']


In [131]:
# Fonction Fitness

A_initial = np.array([[2, 5, 3, 1], 
              [10, 8, 8, 2], 
              [7, 9, 11, 3], 
              [1, 0, 5, 2]])

def fitness(A_initial):
    val = max(A_initial[0]) - min(A_initial[0])
    for i in range(4):
        tempval = max(A_initial[i]) - min(A_initial[i])
        if tempval > val: 
          val = tempval
    return val

print(fitness(A_initial))

8


In [132]:
def get_value_with_position(pos):

    line = pos//4
    colomn = pos%4
    return line, colomn
    
line, colomn = get_value_with_position(11)

In [133]:
# Generate matrix from chromosome 

def generate_matrix_from_chromosome(chromosome, A_initial):
    l = []
    matrice = [[0 for i in range(4)] for j in range(4)]
    for i in range(0, 16):
        a = chromosome[4*i: 4*(i+1)]
        l.append(a)
    
    for i in range(len(l)):
        l[i] = int(l[i], 2)
    
    for i in range(0, 4):
        for j in range(0, 4):
            index_in_l = 4*i+j
            line, column = get_value_with_position(l[index_in_l])
            matrice[i][j] = A_initial[line][column]
    return matrice
    
A = generate_matrix_from_chromosome(population[0], A_initial)
print(A)
    

[[5, 0, 3, 11], [3, 2, 10, 2], [9, 2, 8, 8], [7, 5, 1, 1]]


In [134]:
# Calculate fitness for each chromosome 

def calculate_fitness_for_one_chromosome(chromosome, A_initial):
    chromosome_matrix = generate_matrix_from_chromosome(chromosome, A_initial)
    fitness_of_chromosome = fitness(chromosome_matrix)
    return fitness_of_chromosome


In [135]:
# Calculate fitness for each element of population 
 
def calculate_fitness_each_element_of_population(population, A_initial):
    fitness_of_population = []
    for i in range(len(population)):
        fitness_of_chromosome_i = calculate_fitness_for_one_chromosome(population[i], A_initial)
        fitness_of_population.append(fitness_of_chromosome_i)
    return fitness_of_population
 
fitness_of_population = calculate_fitness_each_element_of_population(population, A_initial)
print(fitness_of_population)

[11, 10, 10, 10]


In [136]:
# Selection new population with the minimal fitness

def get_index_of_minimal(liste):
    a = min(liste)
    for i in range(len(liste)):
        if a == liste[i]:
            return i

def get_selected_population(population, A_initial, number_of_element_to_choose):
    new_population = []
    new_population_fitness = []
    fitness_of_population = calculate_fitness_each_element_of_population(population, A_initial)
    while(number_of_element_to_choose > 0):
        elem = get_index_of_minimal(fitness_of_population)
        new_population.append(population[elem])
        new_population_fitness.append(fitness_of_population[elem])
        fitness_of_population[elem] = 1000000000000000000000000000
        number_of_element_to_choose -= 1
    return new_population, new_population_fitness


new_population, new_population_fitness = get_selected_population(population, A_initial, 3)
print(new_population)
print(new_population_fitness)

['0101101111110010000001110011111010000110101010010100110111000001', '0100000010111101111010010010000111111000001111000111011001011010', '1011101010000110100100111110000011000001001001111101010001011111']
[10, 10, 10]


In [137]:
# Single point crossover operation

def CrossOver_operation(parent1, parent2, point_position):

    child1 = ""
    child2 = ""

    for i in range(point_position):
      child1 += parent1[i]
      child2 += parent2[i]

    for i in range(point_position, len(parent1)):
      child1 += parent2[i]
      child2 += parent1[i]

    
    return child1, child2


child1, child2 = CrossOver_operation(new_population[0], new_population[1], 3)
print(child1)
print(child2)

0100000010111101111010010010000111111000001111000111011001011010
0101101111110010000001110011111010000110101010010100110111000001


In [138]:
# Single point CrossOver for a population

def CrossOver(population, A_initial):

  new_population = []
  point_position = random.choice(np.arange(0, 63))
  for i in range(len(population)):
    for j in range(len(population)):
      if j > i:
        child1, child2 = CrossOver_operation(population[i], population[j], point_position)
        new_population.append(child1)
        new_population.append(child2)
  new_population_fitness = calculate_fitness_each_element_of_population(new_population, A_initial)
  return new_population, new_population_fitness

new_population, new_population_fitness = CrossOver(population, A_initial)
print(len(new_population))
print(new_population_fitness)

12
[11, 6, 11, 10, 11, 8, 9, 10, 10, 10, 10, 9]


In [139]:
# Mutation Operation

mutation_rate_probability = 1/64

def mutation_operation(chromosome, mutation_rate):

  mutated_chromosome = ""

  for i in range(len(chromosome)): 
    if random.random() < mutation_rate_probability:
      mutated_chromosome += str(1 - int(chromosome[i]))
    else:
      mutated_chromosome += chromosome[i]
  return chromosome


mutated = mutation_operation(child1, mutation_rate_probability)

print(mutated)

0100000010111101111010010010000111111000001111000111011001011010


In [140]:
# Mutation for a population 

def mutation(population, mutation_rate, A_initial):

  new_population = []
  new_population_fitness = calculate_fitness_each_element_of_population(population, A_initial)
  for chromosome in population:
    chromosome = mutation_operation(chromosome, mutation_rate)
    new_population.append(chromosome)

  return new_population, new_population_fitness
  
new_population, new_population_fitness = mutation(population, mutation_rate_probability, A_initial)
print(new_population)
print(new_population_fitness)

['1110110110111010001001110100000010011111010101101000000100111100', '0101101111110010000001110011111010000110101010010100110111000001', '0100000010111101111010010010000111111000001111000111011001011010', '1011101010000110100100111110000011000001001001111101010001011111']
[11, 10, 10, 10]


In [141]:
def get_index_min(l):
  minimum = min(l)
  for i in range(len(l)):
    if l[i] == minimum:
      return i

In [144]:
# Genetic Algorithm 

def genetic(matrice, mutation_rate_probability, initial_population_size, taille_selection, ):

  # Population initialisation 
  population = Initialiser_population(population_size=initial_population_size)

  # Fitness of each chromosome
  population_fitness = calculate_fitness_each_element_of_population(population=population, A_initial=matrice)

  while(min(population_fitness) > 9):
    # Selection
    population, population_fitness = get_selected_population(A_initial=matrice, population=population, number_of_element_to_choose=taille_selection)

    # CrossOver
    population, population_fitness = CrossOver(population, matrice )

    # Mutation
    population, population_fitness = mutation(population, mutation_rate_probability, matrice)

  
  index = get_index_min(population_fitness)

  matrix = generate_matrix_from_chromosome(population[index], matrice)

  return matrix


In [148]:
print(matrix)

[[0, 1, 3, 2], [2, 9, 8, 1], [5, 11, 2, 7], [5, 8, 10, 3]]


In [147]:
mutation_rate_probability = 1/64
matrix = genetic(A_initial, mutation_rate_probability, 4, 3)